In [78]:
import argparse
from enum import Enum
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
import time
import random


In [79]:
#Instalar gecko
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service


#Abrir un navegador
from selenium import webdriver
from selenium.webdriver.common.by import By
driver = webdriver.Firefox(service=Service(executable_path=GeckoDriverManager().install()))

In [80]:
URL="https://cpu.userbenchmark.com/"

In [81]:

def waitFinishLoad(driver,idLoader,needsDisapeared=False,by=By.ID,needsWait=True):
    """Espera a la carga de un elemento antes de continuar

    Args:
        driver (WebDriver): WebDriver
        idLoader (str): Id a buscar, puede ser XPath...
        needsDisapeared (bool, optional): Si el elemento a esperar tiene que aparecer o desaparecer como un CircularProgress. Defaults to False.
        by (str, optional): Tipo de By a buscar el Wait. Defaults to By.ID.
        needsWait (bool, optional): Si necesita esperar un extra para que no pueda saltar un Captcha. Defaults to False.
    """
    if idLoader is None:
        return
    WebDriverWait(driver, timeout=60).until(EC.presence_of_element_located((by, idLoader)))
    if needsDisapeared:
        WebDriverWait(driver, timeout=60).until(EC.invisibility_of_element((by, idLoader)))
    if needsWait:
        time.sleep(random.choice([0.05,0.1,0.2]))

def elementBy(query,driver,**kargs):
    """Devuelve un element|None dependiendo de la consulta

    Args:
        query (str): Busqueda a realizar del elemento
        driver (WebDriver): WebDriver

    Returns:
        Element|None: Elemento de la búsqueda
    """
    if not kargs.get("needsWait",False):
        waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    element=driver.find_element(kargs.get("by",By.XPATH),query)
    waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    return element

def clickEvent(driver,element,**kargs):
    """Click en un elemento que funciona con evento

    Args:
        driver (WebDriver): WebDriver
        element (Element): Elemento que tiene el evento
    """
    if not kargs.get("needsWait",False):
        waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    if kargs.get("normal",False):
        element.click
    else:
        try:
            driver.execute_script(element.get_dom_attribute("onClick"))
        except:
            element.click
    waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
# endregion

In [82]:
driver.get(URL)

In [83]:
elementBy("//*[contains(text(),'I understand')]",driver,idLoader="//*[contains(text(),'I understand')]",by=By.XPATH,needsWait=True).click()

In [84]:
from io import StringIO
import pandas as pd
from bs4 import BeautifulSoup


dfT=pd.DataFrame([])
while True:
    try:
        nextB=elementBy("//*[contains(text(),'Next »')]",driver,idLoader="//*[contains(text(),'Next »')]",by=By.XPATH,needsWait=True)
    except:
        break
    soup=BeautifulSoup(driver.page_source,'html.parser')
    df=pd.read_html(StringIO(str(soup.findAll("table")[1])))[0]
    #df=df[~df.iloc[:,0].str.startswith('Página')]
    dfT=pd.concat([dfT,df])
    page=elementBy("//*[contains(text(),'Page ')]",driver,idLoader="//*[contains(text(),'Page ')]",by=By.XPATH,needsWait=True)
    nextB.click()
    waitFinishLoad(driver,idLoader=f"//*[contains(text(),'Page {int(page.text.split(' ')[1])+1}')]",by=By.XPATH)
    

In [85]:
dfT.drop(dfT.columns[0],axis=1,inplace=True)
dfT.drop(dfT.columns[-1],axis=1,inplace=True)

In [86]:
dfT=dfT.rename(columns={dfT.columns[0]:"Name"})

In [87]:
import os
import pathlib

for i in dfT.columns:
    dfT[i] = dfT[i].str.replace("th","")
    dfT[i] = dfT[i].str.replace("nd","")
    dfT[i] = dfT[i].str.replace("rd","")
    dfT[i] = dfT[i].str.replace("st","")
dfT['User rating PTS'] = dfT['User rating  %'].str.split("▼").str[0].str.replace("▲","")
dfT['User rating  %'] = dfT['User rating  %'].str.split("▼").str[1]

In [88]:
dfT

,Name,User rating %,Value %,Avg. bench %,Memory Pts,8-Core Pts,Mkt. share %,Age months,Price,User rating PTS
0,Compare Intel Core i5-13600K €313 Samples 27k,100,11397,122117 - 12899,83.985,145498,0.6197,1795,€31330,103
1,Compare Intel Core i5-12400F €137 Samples 48k,100,11498,9892 - 10594,81.678,97492,1.72100,2793,€13774,98
2,Compare Intel Core i5-12600K €216 Samples 101k,100,11296,109100 - 11797,8074,122396,0.8398,2992,€21651,98
3,Compare Intel Core i3-12100F €96 Samples 11k,100,12199,94.190 - 9992,83.885,78886,0.495,2793,€9686,97
4,Compare Intel Core i7-13700K €404 Samples 41k,100,10993,126120 - 13199,84.787,167699,1.0999,1795,€40421,96
...,...,...,...,...,...,...,...,...,...,...
45,Compare Intel Core2 4400 Samples 1.9k,26,NaN,33.121 - 3913,59.721,40.98,049,121+37,NaN,50
46,Compare Intel Core2 Duo E4400 Samples 1.4k,26,NaN,33.320 - 3813,60.122,41.38,049,122+29,NaN,50
47,Compare Intel Celeron E3200 Samples 981,26,NaN,35.724 - 4319,61.726,60.514,049,120+44,NaN,50
48,Compare AMD Alon X2 240 Samples 291,26,NaN,41.128 - 4628,70.246,7419,049,119+48,NaN,50


In [89]:
dfT.to_csv(os.path.join(str(pathlib.Path().resolve()),"CPU.csv"))